In [ ]:
import tkinter as tk
import tkinter.ttk as ttk
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import (
    FigureCanvasTkAgg, NavigationToolbar2Tk)
import numpy as np

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score,roc_auc_score,roc_curve, auc

import datetime as dt
from mpl_toolkits.mplot3d import Axes3D  #3Dplot
from sklearn import preprocessing #標準化


from datetime import datetime
import scipy

import os
import xlrd
import dask.dataframe as dd
from os import listdir
from tqdm import tqdm_notebook as tqdm
import warnings

# ボタンを押したときの処理 --- (*1)
def calc_regi():
    warnings.filterwarnings("ignore")

    #pd.set_option('display.max_columns', 300)
    #pd.set_option('display.max_rows', 300)
    #各店舗フォルダがあるパスを取得
    #パスはWindows用に別途指定してください。
    path ='C:\\Users\\taban\\Desktop\\マネジメント演習\\本データ\\output'
    folda_path = [filename for filename in listdir(path) if not filename.startswith('.')]
    #'desktop.ini'という要素が発見された。これは要らない。
    #リストから要らない要素を削除する
    folda_path.remove("desktop.ini")
    
    tempo_code = str(text_tempo.get())
    global df_tempo
    global df_raw
    global pd
    
    path = 'C:\\Users\\taban\\Desktop\\マネジメント演習\\本データ\\output\\'+str(text_tempo.get())
    raw = 'C:\\Users\\taban\\Desktop\\マネジメント演習\\本データ\\output\\'+str(text_tempo.get())+'\\{}'
    for i in range(1):#店舗フォルダごとにfor文を回す
        pre_path =path.format(folda_path[i])
        data_path = [filename for filename in listdir(pre_path) if not filename.startswith('.')]
    for j in range(36):#各店舗の月ごとにfor文を回す
        df_raw = pd.read_csv(raw.format(data_path[j]),encoding = "shift_jis")
        if j == 0:
            df_tempo = df_raw
        else:
            df_tempo = pd.concat([df_tempo, df_raw], axis=0)
    
    
     #trainデータ作成
    column_list = ['取引時間', '取引連番','ノードNO']
    df = df_tempo[column_list]
    df['取引時間'] = pd.to_datetime(df['取引時間'])
    df1=df[['取引連番','取引時間']] #曜日タイムダミー出す用
    df2=df.groupby('取引連番').agg({'ノードNO':np.mean,'取引時間':np.max})#それ以外を取引連番毎に結合
    #20日（Tポイント1.5倍）でダミー
    dm_20 = pd.get_dummies(df1['取引時間'].dt.strftime('%d') == '20')
    df1['20日'] = dm_20[True]
    #水曜なら１の水曜ダミーと木曜なら１の木曜日ダミーを作成する
    dm_m = pd.get_dummies(df1['取引時間'].dt.strftime('%a') == 'Mon')
    df1['Mon'] = dm_m[True]
    dm_w = pd.get_dummies(df1['取引時間'].dt.strftime('%a') == 'Wed')
    df1['Wed'] = dm_w[True]
    dm_Th = pd.get_dummies(df1['取引時間'].dt.strftime('%a') == 'Thu')
    df1['Thu'] = dm_Th[True]
    
    #取引連番毎に結合
    newdf1 = df1.groupby('取引連番').agg(np.mean)
    #名前変える
    newdummy = newdf1
    
    df_regi_1 = df2[df2['ノードNO'] == 1] #レジ1のみ抽出
    df_regi_101 = df2[df2['ノードNO'] == 101] #レジ101のみ抽出
    df_regi_102 = df2[df2['ノードNO'] == 102] #レジ102のみ抽出
    df_regi_103 = df2[df2['ノードNO'] == 103] #レジ103のみ抽出
    df_regi_104 = df2[df2['ノードNO'] == 104] #レジ104のみ抽出
    df_regi_105 = df2[df2['ノードNO'] == 105] #レジ105のみ抽出
    #日付ごとの引き算して結合（それぞれで）
    diff_1=df_regi_1['取引時間'].diff()
    regi_1 = pd.concat([df_regi_1, diff_1], axis=1)
    regi_1.columns = ['ノードNO','取引時間','経過時間']
    regi_1['1']=1

    #diffは日付ごとの引き算
    diff_101=df_regi_101['取引時間'].diff()
    regi_101 = pd.concat([df_regi_101, diff_101], axis=1)
    regi_101.columns = ['ノードNO','取引時間','経過時間']
    regi_101['101']=1

    diff_102=df_regi_102['取引時間'].diff()
    regi_102 = pd.concat([df_regi_102, diff_102], axis=1)
    regi_102.columns =  ['ノードNO','取引時間','経過時間']
    regi_102['102']=1

    diff_103=df_regi_103['取引時間'].diff()
    regi_103 = pd.concat([df_regi_103, diff_103], axis=1)
    regi_103.columns = ['ノードNO','取引時間','経過時間']
    regi_103['103']=1

    diff_104=df_regi_104['取引時間'].diff()
    regi_104 = pd.concat([df_regi_104, diff_104], axis=1)
    regi_104.columns =  ['ノードNO','取引時間','経過時間']
    regi_104['104']=1

    diff_105=df_regi_105['取引時間'].diff()
    regi_105 = pd.concat([df_regi_105, diff_105], axis=1)
    regi_105.columns =  ['ノードNO','取引時間','経過時間']
    regi_105['105']=1
    
    #全部合わせる
    regi_all = pd.concat([regi_1, regi_101, regi_102, regi_103, regi_104, regi_105], axis=0)
    train_all = pd.concat([regi_all,newdummy], axis=1)
    #混雑（前後の時間差）が分かればよいため各レジ最初の決済（欠損値）削除
    train_all=train_all.dropna(subset=['経過時間'])
    #経過時間を秒数に変える
    train_all['経過時間']=train_all['経過時間'].dt.seconds

    a=train_all #名前　a　に変更
    #2分ずつ並んだら混雑とみなすためダミー化　各レジで1がどれだけ連続しているかをこれから見る
    a['経過時間'] = pd.get_dummies(a['経過時間']>120)
    
    c = a.groupby(pd.Grouper(key='取引時間',freq='10min')).agg({'経過時間':np.sum,'20日':np.mean,'Mon':np.mean,'Wed':np.mean,'Thu':np.mean,'1':np.mean,'101':np.mean,'102':np.mean,'103':np.mean,'104':np.mean,'105':np.mean}).reset_index()
    c=c.dropna(subset=['20日'])
    c=c.fillna(0)
    c['取引有無'] = pd.Series(c['1']+c['101']+c['102']+c['103']+c['104']+c['105'], index=c.index )
    c['混雑有無']=0
    c.loc[c['経過時間'] >= 3, '混雑有無'] = 1
    c['year']=  c['取引時間'].dt.strftime('%Y').astype(int)
    c['month']= c['取引時間'].dt.strftime('%m').astype(int)
    c['day']= c['取引時間'].dt.strftime('%d').astype(int)
    c['hour']= c['取引時間'].dt.strftime('%H').astype(int)
    c['min']= c['取引時間'].dt.strftime('%M').astype(int)
    
    
    w1 = pd.read_csv('C:\\Users\\taban\\Desktop\\マネジメント演習\\本データ\\気象データ\\つくば市気象データ2017.9-2018.8.csv',encoding = "shift_jis")
    w2 = pd.read_csv('C:\\Users\\taban\\Desktop\\マネジメント演習\\本データ\\気象データ\\つくば市気象データ2018.9-2019.8.csv',encoding = "shift_jis")
    w3 = pd.read_csv('C:\\Users\\taban\\Desktop\\マネジメント演習\\本データ\\気象データ\\つくば市気象データ2019.9-2020.8.csv',encoding = "shift_jis")
    w1 = w1.dropna()
    w2 = w2.dropna()
    w3 = w3.dropna()
    w1 = w1.drop(w1.index[0])
    w2 = w2.drop(w2.index[0])
    w3 = w3.drop(w3.index[0])
    w1 = w1.drop(['Unnamed: 2', 'Unnamed: 3'], axis=1)
    w2 = w2.drop(['Unnamed: 2', 'Unnamed: 3'], axis=1)
    w3 = w3.drop(['Unnamed: 2', 'Unnamed: 3'], axis=1)
    w1.columns = ['日時','天気'] 
    w2.columns = ['日時','天気'] 
    w3.columns = ['日時','天気'] 

    weather = pd.concat([w1,w2,w3], axis=0)
    
    weather['日時'] = pd.to_datetime(weather['日時'])
    weather['year']=  weather['日時'].dt.strftime('%Y').astype(int)
    weather['month']= weather['日時'].dt.strftime('%m').astype(int)
    weather['day']= weather['日時'].dt.strftime('%d').astype(int)
    weather['hour']= weather['日時'].dt.strftime('%H').astype(int)
    #天気が車使うときか否かでダミー化(9-19)
    weather['天気']=weather['天気'].astype(int)
    weather.loc[weather['天気']<9,['天気']]=0
    weather.loc[weather['天気']>19,['天気']]=0
    weather.loc[weather['天気']>=9,['天気']]=1
    weather = weather[['year','month','day','hour','天気']]
    
    #共通列を持つものでweatherとcをまとめる
    sakura_all = pd.merge(c, weather, on=['year', 'month','day','hour'])
    
    d2 = sakura_all.drop(['1','101','102','103','104','105','経過時間'], axis=1)
    
    #「最適開放レジ数」の列を追加
    d2['最適開放レジ数'] = pd.Series(d2['混雑有無']+d2['取引有無'], index=d2.index )
    #列名の変更
    d2.columns = ['取引時間','20日','Mon','Wed','Thu','開放レジ数','混雑したレジの有無','year','month','day','hour','min','天気','最適開放レジ数']
    
    #1hごとに集計
    d2 = d2.groupby(pd.Grouper(key='取引時間',freq='1h')).agg({'20日':np.mean,'Mon':np.mean,'Wed':np.mean,'Thu':np.mean,'hour':np.mean,'天気':np.mean,'最適開放レジ数':np.max}).reset_index()
    #nanを消してダミーを完全にする
    d2=d2.dropna(subset=['20日'])
    d2['20日'] = pd.Series(round(d2['20日']), index=d2.index )
    d2['Mon'] = pd.Series(round(d2['Mon']), index=d2.index )
    d2['Wed'] = pd.Series(round(d2['Wed']), index=d2.index )
    d2['Thu'] = pd.Series(round(d2['Thu']), index=d2.index )
    
    e = d2[(d2['取引時間'] > dt.datetime(2017,9,1))]
    
    
    #trainデータ
    train = e[['hour','20日','Mon','Wed','Thu','天気','最適開放レジ数']]
    
    
    #testデータ作成      
    cols = ['month','day','hour','曜日','天気','最適開放レジ数']
    n = pd.DataFrame(index=[], columns=cols)
    record = pd.Series([float(text_month.get()),float(text_day.get()),0,str(text_youbi.get()),str(text_weather.get()),1], index=n.columns)
    for _ in range(24):
        n = n.append(record, ignore_index=True)
    
    for i in range(24):
        n.loc[i,'hour'] = i

    n['最適開放レジ数']=1
    
    #営業時間外の時間帯を削除
    delete = e[(e['取引時間'] > dt.datetime(2020,8,27)) & (e['取引時間'] <= dt.datetime(2020,8,31))]
    for i in n['hour']:
        x = i in delete['hour'].values
        if x == False:
            n = n[n['hour'] != i]
    
    #行番号を振り直す
    n = n.reset_index()

    #20日（Tポイント1.5倍）でダミー
    n['20日']=0
    n.loc[n['day']=='20',['20日']]=1

    #月水木曜なら１のダミーを作成する
    n['Mon']=0
    n.loc[n['曜日']=='月',['Mon']]=1
    n['Wed']=0
    n.loc[n['曜日']=='水',['Wed']]=1
    n['Thu']=0
    n.loc[n['曜日']=='木',['Thu']]=1


    #天気が車使うときか否かでダミー化(9-19)
    n.loc[n['天気']=='晴れ',['天気']]=0
    n.loc[n['天気']=='曇り',['天気']]=0
    n.loc[n['天気']=='雨',['天気']]=1
    n.loc[n['天気']=='雪',['天気']]=1
    
    #nがtestデータ
    
    #機械学習
    #予測モデルの作成
    y_train=train[['最適開放レジ数']]
    x_train=train[['hour','20日','Mon','Wed','Thu','天気',]]   
    
    #正規化
    sscaler = preprocessing.StandardScaler()
    sscaler.fit(x_train)
    xss_sk = sscaler.transform(x_train) 
    sscaler.fit(y_train)
    yss_sk = sscaler.transform(y_train)
    
    
    from sklearn.ensemble import RandomForestClassifier
    import pandas as pd
  
     
    y_test=n[['最適開放レジ数']]
    x_test=n[['hour','20日','Mon','Wed','Thu','天気']]
    
   
    
    from sklearn.ensemble import RandomForestClassifier
    # 識別モデルの構築
    random_forest = RandomForestClassifier(max_depth=30, n_estimators=30, random_state=42)
    random_forest.fit(x_train, y_train)

    # 予測値算出
    y_pred = random_forest.predict(x_test)    
    
    n['最適開放レジ数予測'] = y_pred
    
    n['20日'] = n['20日'].astype(int)
    n['Mon'] = n['Mon'].astype(int)
    n['Wed'] = n['Wed'].astype(int)
    n['Thu'] = n['Thu'].astype(int)
    n['天気'] = n['天気'].astype(int)
    
    #出力する表の作成
    n2 = n[['hour','最適開放レジ数予測']]
    
    
    # 結果をラベルに表示　ここがデータフレームの出力かな　n2　でもいいかも
    #labelResult['text'] = n2[['hour','最適開放レジ数予測']]
    
    # 結果をラベルに表示
    tree = ttk.Treeview(win)
    # 列を2列作る
    tree["column"] = (1, 2)
    tree["show"] = "headings"
    
    # ヘッダーテキスト
    tree.heading(1, text="hour")
    tree.heading(2, text="最適開放レジ数予測")
    
    # 列の幅
    tree.column(1, width=10)
    tree.column(2, width=10)
    
    # データ挿入
    for i in range(len(n2.index)):
        tree.insert("", "end", values=(n2['hour'][i], n2['最適開放レジ数予測'][i]))
        
    # 設置
    tree.pack(side=tk.TOP, fill=tk.BOTH, expand=1)
    
    
    
    
    root = tk.Tk()#ウインドの作成
    root.title("最適開放レジ数予測")#ウインドのタイトル
    root.geometry("800x600") #ウインドの大きさ

    fig = plt.Figure() #描画の用意
 
    ax = fig.add_subplot(111) 
    
    plt.figure(figsize=(16, 8), dpi=100)
    ax.plot(n2['hour'],n2['最適開放レジ数予測'], marker="o")
    ax.set_title('The hourly optimal number of cashiers')
    ax.set_xlabel('hour')
    ax.set_ylabel('the opitimal number of cashiers')
    

    #tkinterのウインド上部にグラフを表示する
    canvas = FigureCanvasTkAgg(fig, master=root)
    canvas.draw()
    canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)

    #tkinterのウインド下部にツールを追加する
    toolbar = NavigationToolbar2Tk(canvas, root)
    toolbar.update()
    canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)
    
    root.mainloop()
    
    
# ウィンドウを作成 
win = tk.Tk()
win.title("最適開放レジ数予測")
win.geometry("300x780")

# 部品を作成 
label_tempo = tk.Label(win, text=u'店舗名')
label_tempo.pack()

text_tempo= tk.Entry(win)
text_tempo.insert(tk.END, '')
text_tempo.pack()

label_month = tk.Label(win, text=u'〇月')
label_month.pack()

text_month = tk.Entry(win)
text_month.insert(tk.END, '')
text_month.pack()

label_day = tk.Label(win, text=u'〇日')
label_day.pack()

text_day = tk.Entry(win)
text_day.insert(tk.END, '')
text_day.pack()

label_youbi = tk.Label(win, text=u'〇曜日（月～日）')
label_youbi.pack()

text_youbi = tk.Entry(win)
text_youbi.insert(tk.END, '')
text_youbi.pack()

label_weather = tk.Label(win, text=u'天気（晴れor曇りor雨or雪）')
label_weather.pack()

text_weather = tk.Entry(win)
text_weather.insert(tk.END, '')
text_weather.pack()

labelResult = tk.Label(win, text=u'---')
labelResult.pack()

calcButton = tk.Button(win, text=u'出力')
calcButton["command"] = calc_regi
calcButton.pack()



# ウィンドウを動かす
win.mainloop()

1126　つくばみどりの店 9-0

6108　つくばみらい伊奈店 9-22

6124　つくば学園吾妻店 10-0

6113　つくば学園中央店 24h

6272　つくば学園二の宮店 9-0

6278　つくば学園並木店 9-0

6280　つくば吉沼店 24h

6264　つくば研究学園店 24h

6307　つくば研究学園北店 9-0

6349　つくば桜店 24h

6394　つくば春日3丁目店 9-0　（なんか大半のデータないよ）

6245　つくば春日店 9-19

6145　つくば松代店 9-22

6246　つくば大曽根店 9-18

6167　つくば大穂店 24h

6303　つくば谷田部店 9-0

6248　つくば東店 9-0

6181　つくば豊里店 9-21

6165　つくば北条店 9-0

6271　つくば万博記念公園店 9-0